In [5]:
import os
import json
import re
import shutil

# 데이터셋 경로 (training + validation)
data_dirs = [
    # "/data/seungmin/dataset/kr_univ_lec/training/labels/eng",
    "/data/seungmin/dataset/kr_univ_lec/validation/labels/eng"
]

# 원본 wav 파일이 저장된 상위 경로
source_audio_base = "/data/seungmin/dataset/kr_univ_lec/validation/source/eng/arch"

# 저장할 경로
output_dirs = {
    # "train_and": "/data/seungmin/dataset/kr_univ_training_processed/number_and_english",
    "valid_and": "/data/seungmin/dataset/kr_univ_validation_processed/number_and_english",
    # "train_or": "/data/seungmin/dataset/kr_univ_training_processed/number_or_english",
    "valid_or": "/data/seungmin/dataset/kr_univ_validation_processed/number_or_english",
}

# 디렉토리 생성
for path in output_dirs.values():
    os.makedirs(path, exist_ok=True)

# 숫자와 영어 포함 여부 확인 함수
def has_number_and_english(text):
    return bool(re.search(r'\d', text)) and bool(re.search(r'[A-Za-z]', text))

def has_number_or_english(text):
    return bool(re.search(r'[A-Za-z0-9]', text))

# 파일 처리
for data_dir in data_dirs:
    dataset_type = "train" if "training" in data_dir else "valid"

    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.json') and not file.startswith('._'):
                json_path = os.path.join(root, file)

                # JSON 파일 읽기
                with open(json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # 발화 텍스트 가져오기
                text = data.get("06_transcription", {}).get("1_text", "").strip()

                # 상대경로 기반 고유 파일명 생성
                relative_path = os.path.relpath(root, data_dir)
                unique_name = relative_path.replace(os.sep, '_') + '_' + file
                
                # JSON 파일에서 wav 경로 찾기
                wav_filename = file.replace('.json', '.wav')  # JSON과 매칭되는 wav 파일 이름
                lecture_id = relative_path.split(os.sep)[-1]  # Cxxxxx 형태의 강의 ID 추출
                wav_abs_path = os.path.join(source_audio_base, lecture_id, wav_filename)

                # 숫자 AND 영어 포함된 데이터
                if has_number_and_english(text):
                    target_json_path = os.path.join(output_dirs[f"{dataset_type}_and"], unique_name)
                    target_wav_path = os.path.join(output_dirs[f"{dataset_type}_and"], unique_name.replace('.json', '.wav'))

                    shutil.copy2(json_path, target_json_path)
                    if os.path.exists(wav_abs_path):
                        shutil.copy2(wav_abs_path, target_wav_path)

                # 숫자 OR 영어 포함된 데이터 (AND 제외)
                elif has_number_or_english(text):
                    target_json_path_or = os.path.join(output_dirs[f"{dataset_type}_or"], unique_name)
                    target_wav_path_or = os.path.join(output_dirs[f"{dataset_type}_or"], unique_name.replace('.json', '.wav'))

                    shutil.copy2(json_path, target_json_path_or)
                    if os.path.exists(wav_abs_path):
                        shutil.copy2(wav_abs_path, target_wav_path_or)

print("JSON 및 WAV 파일 분류 및 저장 완료.")


✅ JSON 및 WAV 파일 분류 및 저장 완료.
